In [1]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
df = c.data.make_base_df()
df = c.processing.transform_save_imgs(df, c.processing.grayscale_resize)

In [4]:
df

,path,type,filetype,num_id,grayscale_resize_path
0,../data/train/Type_1/0.jpg,Type_1,jpg,0,../data/grayscale_resize/0.jpg
1,../data/train/Type_1/10.jpg,Type_1,jpg,10,../data/grayscale_resize/10.jpg
2,../data/train/Type_1/1013.jpg,Type_1,jpg,1013,../data/grayscale_resize/1013.jpg
3,../data/train/Type_1/1014.jpg,Type_1,jpg,1014,../data/grayscale_resize/1014.jpg
4,../data/train/Type_1/1019.jpg,Type_1,jpg,1019,../data/grayscale_resize/1019.jpg
5,../data/train/Type_1/102.jpg,Type_1,jpg,102,../data/grayscale_resize/102.jpg
6,../data/train/Type_1/1023.jpg,Type_1,jpg,1023,../data/grayscale_resize/1023.jpg
7,../data/train/Type_1/1024.jpg,Type_1,jpg,1024,../data/grayscale_resize/1024.jpg
8,../data/train/Type_1/1026.jpg,Type_1,jpg,1026,../data/grayscale_resize/1026.jpg
9,../data/train/Type_1/1027.jpg,Type_1,jpg,1027,../data/grayscale_resize/1027.jpg


In [ ]:
test_df = c.data.make_test_df()
test_df = c.processing.save_grayscale(df)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC

vecs, types = c.data.random_forest_transform(df)
test_vecs, ids = c.random_forest_transform(test_df, True)
forest = RFC(n_jobs=2,n_estimators=50)

forest.fit(vecs, types)

In [ ]:
preds_probs = forest.predict_proba(test_vecs)

In [ ]:
c.submission.write_submission_file('output.csv', ids, preds_probs)